In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def convert_symbols_to_columns(df, column, value):
    df = df.pivot(columns=column, values=value)
    df2 = pd.DataFrame([])
    df2['date'] = df.index.values
    columns = df.columns
    for column in columns:
        df2[column] = df[column].values
    df2 = df2.set_index('date')
    return df2

In [3]:
def get_data_from_host(url):
    r = requests.get(url)
    return r.json()

In [4]:
project_list_url = 'https://api.debank.com/project/v2/list'
data = get_data_from_host(project_list_url)['data']
for project in data:
    print([project['id'], project['name']['en']])

['0x', '0x']
['1inch', '1inch V1']
['1inch2', '1inch']
['1sd', '1sd']
['88mph', '88mph V1']
['88mph2', '88mph V2']
['aave', 'Aave V1']
['aave2', 'Aave V2']
['aavegotchi', 'Aavegotchi']
['aave_amm', 'Aave AMM']
['acbtc', 'acBTC']
['aco', 'ACO']
['airswap', 'AirSwap']
['alchemix', 'Alchemix']
['alpaca', 'Alpaca City']
['alpha', 'Alpha Homora']
['alpha2', 'Alpha Homora V2']
['ampleforth', 'Ampleforth']
['anyswap', 'AnySwap']
['apwine', 'APWine']
['apy', 'APY.Finance']
['aragon', 'Aragon']
['arcx', 'ARCx']
['armor', 'Armor']
['artem', 'Artem']
['async_art', 'Async Art']
['augur', 'Augur']
['badger', 'Badger DAO']
['balancer', 'Balancer']
['balancer2', 'Balancer V2']
['bancor', 'Bancor']
['band', 'Band']
['barnbridge', 'BarnBridge']
['base', 'Base']
['basiscash', 'Basis Cash']
['basiscoin', 'Basis Coin']
['basisdollar', 'Basis Dollar']
['basisgold', 'Basis Gold']
['bdp', 'Big Data Protocol']
['bella', 'Bella']
['benchmark', 'Pendle']
['bonded', 'Bonded']
['boringdao', 'BoringDAO']
['bounce'

In [5]:
id_name = {"badger" : "badger dao"}
# for row in data:
#     id_name[row['id']] = row['name']['en']

In [14]:
tvl_df = pd.DataFrame
for project_id in id_name:
    project_id = "bsc_venus"
    uri = f"https://api.debank.com/project/tvl_chart?id={project_id}"
    print(uri)
    data = get_data_from_host(uri)
    data = data['data']
    df = pd.DataFrame(data)
    if not df.empty:
        df = df[['date_at', 'locked_usd_value']].copy()
        df['project'] = id_name[project_id]
        if tvl_df.empty:
            tvl_df = df
        else:
            tvl_df = tvl_df.append(df)
    break
# tvl_df = tvl_df.set_index(['date_at'])

https://api.debank.com/project/tvl_chart?id=bsc_venus


In [15]:
len(tvl_df['project'].unique())

1

In [16]:
df

,date_at,locked_usd_value,project
0,2020-11-23,8.362049e+05,Venus
1,2020-11-24,8.805606e+06,Venus
2,2020-11-25,3.781105e+07,Venus
3,2020-11-26,1.138477e+08,Venus
4,2020-11-27,1.076491e+08,Venus
...,...,...,...
172,2021-05-14,9.087139e+09,Venus
173,2021-05-15,8.305369e+09,Venus
174,2021-05-16,8.366557e+09,Venus
175,2021-05-17,7.036477e+09,Venus


In [12]:
tvl_df[tvl_df['project'] == 'Venus']

,date_at,locked_usd_value,project


In [14]:
tvl_df = convert_symbols_to_columns(tvl_df, 'project', 'locked_usd_value')

In [11]:
len(tvl_df.columns.tolist())

NameError: name 'tvl_df' is not defined

In [6]:
def get_json_data(url, columns):
    r = requests.get(url)
    print(r)
    data = r.json()['data']
    if 'data' in data and len(data['data']):
        df = pd.DataFrame(data['data'])
        df.columns = columns
        return df
    else:
        now = datetime.now().strftime("%Y-%m-%d")
        return pd.DataFrame([[now, np.nan]], columns=columns)
    

In [7]:
tvl_url = 'https://api.debank.com/project/chart?id={id}&type=tvl'
borrow_url = 'https://api.debank.com/project/chart?id={id}&type=borrow'
trade_user_url = 'https://api.debank.com/project/chart?id={id}&type=trade_user'
liquidate_url = 'https://api.debank.com/project/chart?id={id}&type=liquidate'
trade_volume_url = 'https://api.debank.com/project/chart?id={id}&type=trade_volume'
trade_count_url = 'https://api.debank.com/project/chart?id={id}&type=trade_count'
oncall_contract_url = 'https://api.debank.com/project/chart?id={id}&type=oracle_contract'
oncall_call_url = 'https://api.debank.com/project/chart?id={id}&type=oracle_call'

In [8]:
def get_data_df_joined_to(join_df, url, data_name):
    print(url, data_name)
    columns = ['date', data_name]
    df = get_json_data(url, columns = columns)
    if not df.empty:
        df.index = pd.DatetimeIndex(df['date'])
        del df['date']
    elif not join_df.empty:
        join_df[data_name] = np.nan
        return join_df
    if join_df.empty:
        return df
    else:
#         join_df = join_df.(df)
        join_df = pd.merge(join_df, df, how='outer')
        return join_df
        

In [9]:
# for p_id in id_name:
def get_debank_project_data_df(p_id):
    join_df = pd.DataFrame(columns=['tvl', 'borrow', 'trade_users', 'trade_volume', 'trade_count', 'oncall_contract', 'oncall_call'])
    join_df = get_data_df_joined_to(join_df, tvl_url.format(id = p_id), 'tvl')
    join_df = get_data_df_joined_to(join_df, borrow_url.format(id = p_id), 'borrow')
    join_df = get_data_df_joined_to(join_df, trade_user_url.format(id = p_id), 'trade_users')
#     join_df = get_data_df_joined_to(join_df, liquidate_url.format(id = p_id), 'liquidate')
    join_df = get_data_df_joined_to(join_df, trade_volume_url.format(id = p_id), 'trade_volume')
    join_df = get_data_df_joined_to(join_df, trade_count_url.format(id = p_id), 'trade_count')
    join_df = get_data_df_joined_to(join_df, oncall_contract_url.format(id = p_id), 'oncall_contract')
    join_df = get_data_df_joined_to(join_df, oncall_call_url.format(id = p_id), 'oncall_call')
    join_df = join_df.reset_index()
    return join_df


In [10]:
from time import sleep
df = pd.DataFrame
for p_id in id_name:
    sleep(1)
    print([p_id, id_name[p_id]])
    df2 = get_debank_project_data_df(p_id)
    if not df2.empty:
        df2['project'] = id_name[p_id]
    if df.empty:
        df = df2
    else:
        df = df.append(df2)

df = df.drop_duplicates()

['badger', 'badger dao']
https://api.debank.com/project/chart?id=badger&type=tvl tvl
<Response [200]>
https://api.debank.com/project/chart?id=badger&type=borrow borrow
<Response [200]>


MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [54]:
df[df['project'].isin(['Aave V1'])]

,date,tvl,borrow,trade_users,trade_volume,trade_count,oncall_contract,oncall_call,project
0,2020-01-08,3.289397e+05,1.356566e+03,NaN,NaN,NaN,NaN,NaN,Aave V1
1,2020-01-09,4.926138e+05,5.977061e+04,NaN,NaN,NaN,NaN,NaN,Aave V1
2,2020-01-10,8.255981e+05,1.821626e+05,NaN,NaN,NaN,NaN,NaN,Aave V1
3,2020-01-11,9.949300e+05,2.649026e+05,NaN,NaN,NaN,NaN,NaN,Aave V1
4,2020-01-12,1.196295e+06,3.195177e+05,NaN,NaN,NaN,NaN,NaN,Aave V1
...,...,...,...,...,...,...,...,...,...
460,2021-04-12,3.091809e+09,6.425478e+08,NaN,NaN,NaN,NaN,NaN,Aave V1
461,2021-04-13,3.157051e+09,6.377181e+08,NaN,NaN,NaN,NaN,NaN,Aave V1
462,2021-04-14,3.371132e+09,6.413465e+08,NaN,NaN,NaN,NaN,NaN,Aave V1
463,2021-04-15,3.513497e+09,6.440395e+08,NaN,NaN,NaN,NaN,NaN,Aave V1


In [55]:
df[df['project'].isin(['Aave V2'])]

,date,tvl,borrow,trade_users,trade_volume,trade_count,oncall_contract,oncall_call,project
0,2020-11-30,1.234000e-01,NaN,NaN,NaN,NaN,NaN,NaN,Aave V2
1,2020-12-01,1.057623e+01,9.980090e-01,NaN,NaN,NaN,NaN,NaN,Aave V2
2,2020-12-02,1.137998e+04,2.700363e+02,NaN,NaN,NaN,NaN,NaN,Aave V2
3,2020-12-03,2.957170e+06,7.446809e+05,NaN,NaN,NaN,NaN,NaN,Aave V2
4,2020-12-04,1.427062e+07,1.433082e+06,NaN,NaN,NaN,NaN,NaN,Aave V2
...,...,...,...,...,...,...,...,...,...
133,2021-04-12,2.952956e+09,8.093778e+08,NaN,NaN,NaN,NaN,NaN,Aave V2
134,2021-04-13,3.046765e+09,8.215748e+08,NaN,NaN,NaN,NaN,NaN,Aave V2
135,2021-04-14,3.251545e+09,8.296486e+08,NaN,NaN,NaN,NaN,NaN,Aave V2
136,2021-04-15,3.394313e+09,8.374277e+08,NaN,NaN,NaN,NaN,NaN,Aave V2


In [56]:
df[df['project'].isin(['0x'])]

,date,tvl,borrow,trade_users,trade_volume,trade_count,oncall_contract,oncall_call,project
0,2021-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x


In [196]:
import pymysql
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_debank_projects_data_df(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        try:
            print(dt, row["project"], row["tvl"], row["trade_users"], row["trade_volume"], row["trade_count"], row["oncall_contract"], row["oncall_call"], row['borrow'])
            sql = "INSERT INTO `debank_project_data` (`timestamp_dt`, `project`, `tvl`, `trading_users`, `trading_volume`, `transactions`, `call_contract`, `number_of_calls`, `borrow`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["project"], row["tvl"], row["trade_users"], row["trade_volume"], row["trade_count"], row["oncall_contract"], row["oncall_call"], row['borrow']))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [197]:
df.columns

Index(['date', 'tvl', 'borrow', 'trade_users', 'trade_volume', 'trade_count',
       'oncall_contract', 'oncall_call', 'project'],
      dtype='object')

In [198]:
df = df.astype(object).where(pd.notnull(df), None)
insert_debank_projects_data_df(df)

1588636800.0 0x 1.99686156666356 2.0 0.011 2.0 None None None
(1062, "Duplicate entry '2020-05-05 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1588723200.0 0x 4.224535088477372 2.0 0.0024000000000000002 3.0 None None None
(1062, "Duplicate entry '2020-05-06 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1588809600.0 0x 4.317138142419149 None None None None None None
(1062, "Duplicate entry '2020-05-07 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1588896000.0 0x 25.383476506048233 None None None None None None
(1062, "Duplicate entry '2020-05-08 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1588982400.0 0x 25.243735300199763 None None None None None None
(1062, "Duplicate entry '2020-05-09 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1589068800.0 0x 23.88337322339844 None None None None None None
(1062, "Duplicate entry '2020-05-10 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1589155200.0 0x 23.88651728539254 2.0 0.397523 3.0 None None None
(

(1062, "Duplicate entry '2020-06-23 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1592956800.0 0x 33014233.40162418 4090.0 13582015.273438683 17470.0 None None None
(1062, "Duplicate entry '2020-06-24 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1593043200.0 0x 31202155.473449416 3588.0 13744772.339961577 15640.0 None None None
(1062, "Duplicate entry '2020-06-25 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1593129600.0 0x 32341236.279671803 3677.0 12619200.151604792 17273.0 None None None
(1062, "Duplicate entry '2020-06-26 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1593216000.0 0x 33561222.03841641 3659.0 13000252.666372385 16456.0 None None None
(1062, "Duplicate entry '2020-06-27 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1593302400.0 0x 34013908.50796709 3662.0 8560534.170667378 16388.0 None None None
(1062, "Duplicate entry '2020-06-28 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1593388800.0 0x 37131357.690393515 3809.0 9274468.

(1062, "Duplicate entry '2020-08-12 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597276800.0 0x 260838834.40114075 19263.0 172390482.77153492 65166.0 5.0 338.0 None
(1062, "Duplicate entry '2020-08-13 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597363200.0 0x 246012776.30826393 19312.0 153186514.797046 65116.0 4.0 131.0 None
(1062, "Duplicate entry '2020-08-14 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597449600.0 0x 251996263.0000492 18439.0 124900212.90069252 63032.0 3.0 130.0 None
(1062, "Duplicate entry '2020-08-15 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597536000.0 0x 253476553.58826303 20891.0 128461750.34370622 74011.0 3.0 194.0 None
(1062, "Duplicate entry '2020-08-16 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597622400.0 0x 259594669.87986827 20304.0 136421304.73314664 68270.0 3.0 160.0 None
(1062, "Duplicate entry '2020-08-17 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1597708800.0 0x 279149708.3988853 23852.0 17

(1062, "Duplicate entry '2020-09-30 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601510400.0 0x 2408062726.2173204 28753.0 254054798.8744235 86665.0 13.0 757.0 None
(1062, "Duplicate entry '2020-10-01 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601596800.0 0x 2376377977.5594397 29632.0 289821103.60779405 91671.0 16.0 819.0 None
(1062, "Duplicate entry '2020-10-02 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601683200.0 0x 2409508315.280655 28016.0 189119230.788529 89790.0 19.0 1216.0 None
(1062, "Duplicate entry '2020-10-03 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601769600.0 0x 2445505096.585678 30565.0 216178690.8156993 105563.0 12.0 636.0 None
(1062, "Duplicate entry '2020-10-04 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601856000.0 0x 2506968519.876069 30650.0 191179766.93607768 104457.0 16.0 674.0 None
(1062, "Duplicate entry '2020-10-05 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1601942400.0 0x 2430552822.543481 32441.

(1062, "Duplicate entry '2020-11-17 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1605657600.0 0x 1595966252.0727344 23997.0 219866320.942512 71686.0 38.0 5838.0 None
(1062, "Duplicate entry '2020-11-18 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1605744000.0 0x 1566671057.3996618 25849.0 189232358.02060688 80789.0 40.0 5721.0 None
(1062, "Duplicate entry '2020-11-19 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1605830400.0 0x 1632866643.1909633 25892.0 224816726.93853608 82492.0 43.0 5218.0 None
(1062, "Duplicate entry '2020-11-20 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1605916800.0 0x 1649231964.874986 25496.0 288369093.9176865 87301.0 42.0 5666.0 None
(1062, "Duplicate entry '2020-11-21 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1606003200.0 0x 1666666505.9708507 25546.0 335348783.0756381 87610.0 39.0 6235.0 None
(1062, "Duplicate entry '2020-11-22 00:00:00-0x' for key 'debank_project_data.PRIMARY'")
1606089600.0 0x 1709190881.4724426 2

...

1600473600.0 Aave AMM 2150513697.9937844 32071.0 345299897.45547277 93931.0 13.0 410.0 None
1600560000.0 Aave AMM 2169389143.9146466 30457.0 273488463.8966453 90407.0 10.0 273.0 None
1600646400.0 Aave AMM 1974687896.3191707 29186.0 329169113.86688185 87454.0 14.0 432.0 None
1600732800.0 Aave AMM 1986810792.375272 32073.0 231712354.65136036 97933.0 13.0 1956.0 None
1600819200.0 Aave AMM 1905938532.3224092 30204.0 246521073.62127215 93929.0 12.0 1833.0 None
1600905600.0 Aave AMM 2068748061.8427277 28236.0 194354338.43586108 83808.0 18.0 1082.0 None
1600992000.0 Aave AMM 2152693190.368544 29731.0 182963539.7265199 87971.0 17.0 995.0 None
1601078400.0 Aave AMM 2180642440.724249 27004.0 170922809.1805799 84939.0 13.0 586.0 None
1601164800.0 Aave AMM 2248485273.5085444 27438.0 204562762.6978456 86232.0 14.0 649.0 None
1601251200.0 Aave AMM 2290110020.9562087 30092.0 205733131.40516534 90661.0 16.0 724.0 None
1601337600.0 Aave AMM 2333894666.496564 31053.0 218124221.2465721 89577.0 15.0 1279.

1608249600.0 Aave AMM 1820155859.8956132 32761.0 319519323.1413073 102376.0 48.0 7451.0 None
1608336000.0 Aave AMM 1824764984.389773 33363.0 334026888.7662035 115728.0 39.0 6696.0 None
1608422400.0 Aave AMM 1802668216.8236468 35700.0 323159397.8921988 127438.0 47.0 6949.0 None
1608508800.0 Aave AMM 1836553998.9836667 35596.0 401579565.80768114 134463.0 57.0 6846.0 None
1608595200.0 Aave AMM 1901195238.7384176 32631.0 357972729.0000158 110387.0 54.0 7445.0 None
1608681600.0 Aave AMM 1879784714.454703 31552.0 390819258.57042193 103242.0 46.0 6673.0 None
1608768000.0 Aave AMM 1979016267.3221452 29964.0 358181454.87152594 98127.0 52.0 6978.0 None
1608854400.0 Aave AMM 2008857119.017953 29162.0 341635483.42195165 101739.0 47.0 6669.0 None
1608940800.0 Aave AMM 1984774992.5710535 28415.0 333350530.587675 99108.0 53.0 7003.0 None
1609027200.0 Aave AMM 2005981810.0719016 29247.0 475512595.90304613 103462.0 51.0 6938.0 None
1609113600.0 Aave AMM 2055974687.9089363 33597.0 448687515.44508123 112

1616112000.0 Aave AMM 5249261726.829461 51409.0 917571937.2179236 140882.0 41.0 5985.0 None
1616198400.0 Aave AMM 5244917941.5385685 53320.0 849859136.2537634 140916.0 33.0 5890.0 None
1616284800.0 Aave AMM 5183978123.194017 48627.0 749583637.0388856 133220.0 34.0 6452.0 None
1616371200.0 Aave AMM 4996771738.257225 54085.0 1159630401.2588196 151476.0 43.0 11705.0 None
1616457600.0 Aave AMM 4869252744.03758 51864.0 949254365.7544754 140672.0 38.0 11013.0 None
1616544000.0 Aave AMM 4736639384.701706 51909.0 1087938055.694412 146691.0 35.0 11644.0 None
1616630400.0 Aave AMM 4685355550.19025 47476.0 941546532.9770331 130886.0 44.0 7059.0 None
1616716800.0 Aave AMM 5013602651.4481 49693.0 853528375.9814194 138199.0 40.0 5482.0 None
1616803200.0 Aave AMM 5083063015.58234 50596.0 883790036.7583969 146172.0 37.0 6073.0 None
1616889600.0 Aave AMM 5084946934.396577 49538.0 801592261.8478227 145607.0 37.0 5708.0 None
1616976000.0 Aave AMM 5291627690.486383 50589.0 1017106123.185721 146401.0 42.0 

1594771200.0 acBTC 61549600.17597338 5241.0 19484068.270336747 18371.0 None None None
1594857600.0 acBTC 64097599.65186608 4984.0 18873224.817195084 17556.0 None None None
1594944000.0 acBTC 65625034.57901766 6878.0 28278342.147584274 24901.0 1.0 8.0 None
1595030400.0 acBTC 68288354.41612993 7138.0 29945518.182137266 27876.0 None None None
1595116800.0 acBTC 72576857.96935385 8340.0 32970106.688121643 34104.0 None None None
1595203200.0 acBTC 72423213.53242 8466.0 36535122.341367416 32131.0 None None None
1595289600.0 acBTC 75638908.52751298 8597.0 34734789.10972421 30680.0 1.0 1.0 None
1595376000.0 acBTC 80853167.52173738 9104.0 39834348.564869195 33159.0 None None None
1595462400.0 acBTC 82888226.66655022 8920.0 47294585.327406295 31394.0 1.0 1.0 None
1595548800.0 acBTC 88800608.27767989 9328.0 52410051.94269814 34041.0 None None None
1595635200.0 acBTC 101972246.58226866 10228.0 68489793.0855086 38811.0 None None None
1595721600.0 acBTC 109580334.87364016 10503.0 71154886.03532559 3

KeyboardInterrupt: 

In [39]:
dex_histori_api='https://api.debank.com/old/dex/histories'
dex_chart_api='https://api.debank.com/old/dex/charts'

In [40]:
data = get_data_from_host(dex_histori_api)['data']

In [10]:
dex_df = pd.DataFrame
for row in data:
    df = pd.DataFrame(row['dates'])
    if dex_df.empty:
        dex_df = df
    else:
        dex_df = dex_df.append(df)

In [11]:
dex_df = dex_df[['active_user_count','amount', 'count', 'project_id','date_at']]
dex_df = dex_df.set_index(['date_at'])
# dex_df = convert_symbols_to_columns(dex_df, 'project_id', ['active_user_count','amount','count'])

In [12]:
dex_df

,active_user_count,amount,count,project_id
date_at,,,,
2021-01-28,12,9.419078e+05,15,1inch
2020-06-19,548,3.133298e+07,1368,1inch
2020-01-12,97,2.548992e+05,256,1inch
2020-11-02,460,3.131499e+07,788,1inch
2020-03-07,199,1.766560e+06,589,1inch
...,...,...,...,...
2021-02-23,5,5.722544e+04,28,yfv
2021-02-04,7,1.679056e+05,95,yfv
2021-02-20,7,1.792941e+05,61,yfv


In [57]:
stable_coin_api='https://api.debank.com/stable_coins'
stable_total_supply_dates_api='https://api.debank.com/stable_coins/total_supply/date'

In [59]:
data = get_data_from_host(stable_coin_api)['data']

In [60]:
data2 = get_data_from_host(stable_total_supply_dates_api)['data']

In [61]:
stable_df = pd.DataFrame
id_symbol = {}
for row in data:
    id_symbol[row['id']] = row['symbol']

for key in data2:
    key_data = data2[key]
    symbol = id_symbol[key]
    df = pd.DataFrame(key_data)
    df['symbol'] = symbol
    if stable_df.empty:
        stable_df = df
    else:
        stable_df = stable_df.append(df)
stable_df = stable_df.set_index(['date_at'])

In [37]:
stable_df = convert_symbols_to_columns(stable_df, 'symbol', 'value')

In [62]:
stable_df

,value,symbol
date_at,,
2018-03-05,4.761925e+06,TUSD
2018-03-06,4.761925e+06,TUSD
2018-03-07,4.761925e+06,TUSD
2018-03-08,4.761925e+06,TUSD
2018-03-09,6.561925e+06,TUSD
...,...,...
2021-02-27,4.209304e+06,USDx
2021-02-28,4.209304e+06,USDx
2021-03-01,4.209304e+06,USDx


In [63]:
# lending_id_api = f'https://api.debank.com/debt/compound/borrow_amount/date'
lending_api='https://api.debank.com/debt/borrow_amount/date'

In [64]:
data = get_data_from_host(lending_api)['data']

In [65]:
lending_df = pd.DataFrame
for key in data:
    key_data = data[key]
#     lending_id_api = f'https://api.debank.com/debt/{key}/borrow_amount/date'
#     key_data = get_data_from_host(lending_id_api)['data']
    df = pd.DataFrame(key_data)
    df['symbol'] = key
    if lending_df.empty:
        lending_df = df
    else:
        lending_df = lending_df.append(df)
lending_df = lending_df.set_index(['date_at'])
# lending_df = convert_symbols_to_columns(lending_df, 'symbol', 'value')

In [66]:
lending_df

,value,symbol
date_at,,
2020-01-08,1.352951e+03,aave
2020-01-09,5.989081e+04,aave
2020-01-10,1.822150e+05,aave
2020-01-11,2.650576e+05,aave
2020-01-12,3.183697e+05,aave
...,...,...
2021-02-27,7.765190e+06,wepiggy
2021-02-28,7.292302e+06,wepiggy
2021-03-01,8.364142e+06,wepiggy


In [43]:
btc_api='https://api.debank.com/token/btc_pegged_list'
# data = get_data_from_host(btc_api)['data']

In [44]:
oracle_api='https://api.debank.com/oracle'
data = get_data_from_host(oracle_api)['data']

In [45]:
oracle_call_contracts_df = pd.DataFrame(data)
oracle_call_contracts_df

,call_count,contract_count,id,is_show,update_at
0,6837,75,chainlink,True,2021-02-27 11:45:41
1,362,5,makerdao,True,2021-02-27 11:45:41
2,14,1,nest,True,2021-02-27 11:45:41
3,2576,4,sushiswap,True,2021-02-27 11:45:41
4,5021,33,uniswap2,True,2021-02-27 11:45:41


In [46]:
oracle_date_api = 'https://api.debank.com/oracle/call_count/date'

In [47]:
data = get_data_from_host(oracle_date_api)['data']

In [48]:
oracle_call_count_df = pd.DataFrame
for key in data:
    key_data = data[key]
    df = pd.DataFrame(key_data)
    df['symbol'] = key
    if oracle_call_count_df.empty:
        oracle_call_count_df = df
    else:
        oracle_call_count_df = oracle_call_count_df.append(df)
oracle_call_count_df = oracle_call_count_df.set_index(['date_at'])
oracle_call_count_df = convert_symbols_to_columns(oracle_call_count_df, 'symbol', 'value')
oracle_call_count_df

,chainlink,makerdao,nest,sushiswap,uniswap2
date,,,,,
2019-11-13,NaN,2.0,NaN,NaN,NaN
2019-11-15,NaN,46.0,NaN,NaN,NaN
2019-11-16,NaN,96.0,NaN,NaN,NaN
2019-11-17,NaN,96.0,NaN,NaN,NaN
2019-11-18,NaN,92.0,NaN,NaN,NaN
...,...,...,...,...,...
2021-02-23,13271.0,443.0,13.0,2476.0,5368.0
2021-02-24,8558.0,387.0,15.0,2508.0,5878.0
2021-02-25,6028.0,362.0,15.0,2604.0,5264.0


In [11]:
len(['1INCH', 'AAVE', 'ACS', 'ADD', 'aEth', 'ALCX', 'ALPA', 'ALPACA', 'ALPHA', 'AMP', 'AMPL', 'ANT', 'ANY', 'APY', 'ARMOR', 'AST', 'AUCTION', 'AUTO', 'BAC', 'BADGER', 'BAG', 'BAKE', 'BAL', 'BAND', 'BANK', 'BAS', 'BASE', 'BCS', 'BDO', 'BDP', 'BEL', 'BELT', 'BNT', 'BOND', 'BOR', 'BSD', 'BSDS', 'BSGS', 'BSWAP', 'BT', 'BUNNY', 'BZRX', 'CAKE', 'COFI', 'COMBO', 'COMP', 'CORE', 'COVER', 'CREAM', 'CRV', 'CVP', 'DAIQ', 'DDIM', 'DDX', 'DEFI++', 'DEGO', 'DEGOV', 'DEXE', 'DEXTF', 'DF', 'DFD', 'DFT', 'DHT', 'DMG', 'DODO', 'DOKI', 'DOLLAR', 'DOS', 'DOUGH', 'DRC', 'DSD', 'DUCK', 'EGG', 'EPS', 'EQZ', 'ESD', 'eXRD', 'FARM', 'FEI', 'FIN', 'FL', 'FLX', 'FNX', 'FOR', 'FRAX', 'FRM', 'FSAFE', 'FXC', 'FXS', 'GET', 'GHST', 'GNO', 'GOF', 'GRO', 'GYSR', 'HAKKA', 'HEGIC', 'HELMET', 'HGET', 'HOT', 'ICHI', 'IDEX', 'IDLE', 'INDEX', 'INV', 'JRT', 'JULD', 'KEEP', 'KFC', 'KIMCHI', 'KNC', 'KP3R', 'KUN', 'L2', 'LDO', 'LEV', 'LIEN', 'LINA', 'LINK', 'LON', 'LRC', 'LUA', 'MARK', 'Mars', 'MCB', 'MEE', 'MEME', 'MIC', 'MINI', 'MIR', 'MIS', 'MIS2', 'MKR', 'MLN', 'MM', 'MONA', 'MPH\n', 'MPH', 'MTA', 'N3RDz', 'NDX', 'NEC', 'NEST', 'NFTFY', 'NRV', 'Nsure', 'NXM', 'ONC', 'ONES', 'ONS', 'OPEN', 'OPIUM', 'OPT', 'ORAI', 'ORN', 'PBTC', 'PCT', 'PEAK', 'PERL', 'PERP', 'PHOON', 'PICKLE', 'PLOT', 'PNT', 'POOL', 'POWER', 'RAI', 'RAMP', 'RAY', 'RBBT', 'REN', 'REP', 'REPv2', 'RF!', 'RFI', 'RGT', 'ROOK', 'ROOT', 'RSR', 'RSV', 'SAC', 'SAFE', 'SAFE2', 'SAKE', 'SAS', 'SASHIMI', 'SDT', 'SFG', 'SFI', 'SHELL', 'SI', 'SIG', 'SNOW', 'SNX', 'STAKE', 'STBZ', 'STKR', 'STN', 'STONE', 'SUSHI', 'SWAP', 'SWINGBY', 'SWISS', 'SWRV', 'SWTH', 'SYN', 'TBTC', 'TEAM', 'TORN', 'TRB', 'TRU', 'UMA', 'UMB', 'UMX', 'UNC', 'UNCX', 'UNI', 'UNN', 'USF', 'VALUE', 'VSP', 'WARP', 'wxDai', 'xBTC', 'XDEX', 'XOS', 'XSAT', 'XTK', 'XUSD', 'XVIX', 'XVS', 'YAM', 'YAX', 'YELD', 'YF-DAI', 'YFI', 'YFII', 'YFIII', 'YFL', 'YIELDX', 'YVS', 'ZAI', 'ZKS', 'ZRX', 'AUC', 'CRX', 'DEFI', 'FST', 'IMX', 'LIGHT', 'LQTY', 'MATIC', 'MET', 'NXS', 'ONX', 'PIPT', 'ROBO', 'TRADE', 'AC', 'BANANA', 'BHSC', 'BIFI', 'BTCST', 'CCAKE', 'DFX', 'DND', 'FUEL', 'KEBAB', 'MDO', 'MDX', 'NAR', 'NUTS', 'PMP', 'RUPEE', 'SEED', 'SMTY', 'UP', 'WHEAT', 'XEND', 'ADA', 'AR', 'BASK', 'BNB', 'BTC', 'CFG', 'COL', 'DCR', 'DOGE', 'DOT', 'ETH', 'FIL', 'KAVA', 'KSM', 'LPT', 'LTC', 'LUNA', 'NMR', 'PNG', 'QUICK', 'RARI', 'RUNE', 'SRM', 'TRX', 'XLM', 'XTZ', 'ZEC', 'ALEPH', 'ANCT', 'AUDIO', 'FIDA', 'FRONT', 'GRT', 'KIN', 'MANGO', 'MAPS', 'ORCA', 'OXY', 'PRQ', 'SOL', 'STEP', 'USDC', 'USDT', 'WAVES'])

323